<a href="https://colab.research.google.com/github/RyanChen12035/capstone/blob/main/Llama_2_13B_GGML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Llama 2**

The Llama 2 is a collection of pretrained and fine-tuned generative text models, ranging from 7 billion to 70 billion parameters, designed for dialogue use cases.

 It outperforms open-source chat models on most benchmarks and is on par with popular closed-source models in human evaluations for helpfulness and safety.

[Llama 2 13B-chat](https://huggingface.co/meta-llama/Llama-2-13b-chat)

`llama.cpp`'s objective is to run the LLaMA model with 4-bit integer quantization on MacBook. It is a plain C/C++ implementation optimized for Apple silicon and x86 architectures, supporting various integer quantization and BLAS libraries. Originally a web chat example, it now serves as a development playground for ggml library features.

`GGML`, a C library for machine learning, facilitates the distribution of large language models (LLMs). It utilizes quantization to enable efficient LLM execution on consumer hardware. GGML files contain binary-encoded data, including version number, hyperparameters, vocabulary, and weights. The vocabulary comprises tokens for language generation, while the weights determine the LLM's size. Quantization reduces precision to optimize resource usage.

#  Quantized Models from the Hugging Face Community

The Hugging Face community provides quantized models, which allow us to efficiently and effectively utilize the model on the T4 GPU. It is important to consult reliable sources before using any model.

There are several variations available, but the ones that interest us are based on the GGLM library.

We can see the different variations that Llama-2-13B-GGML has [here](https://huggingface.co/models?search=llama%202%20ggml).



In this case, we will use the model called [Llama-2-13B-chat-GGML](https://huggingface.co/TheBloke/Llama-2-13B-chat-GGML).

#**Step 1: Install All the Required Packages**

In [1]:
# GPU llama-cpp-python
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.78 numpy==1.23.4 --force-reinstall --upgrade --no-cache-dir --verbose
!pip install huggingface_hub
!pip install llama-cpp-python==0.1.78
!pip install numpy==1.23.4

Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 21.6 MB/s eta 0:00:00
  Running command pip subprocess to install build dependencies
  Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.3/819.3 kB 15.5 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 12.7 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.3/26.3 MB 62.2 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 39.1 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 8.0 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.4/65.4 kB 10.1 MB/s eta 0:00:00
    Creating /tmp/pip-build-env-djl8pw34/overlay/local/bin
    changing mode of /tmp/pip-build-env-djl8pw34/overlay/local/bin/ninja to 755
    changing mode of /tmp/pip-build-env-djl8pw34/overlay

In [2]:
model_name_or_path = "TheBloke/Llama-2-13B-chat-GGML"
model_basename = "llama-2-13b-chat.ggmlv3.q5_1.bin" # the model is in bin format

#**Step 2: Import All the Required Libraries**

In [3]:
from huggingface_hub import hf_hub_download


In [4]:
from llama_cpp import Llama


#**Step 3: Download the Model**

In [5]:
model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


llama-2-13b-chat.ggmlv3.q5_1.bin:   0%|          | 0.00/9.76G [00:00<?, ?B/s]

#**Step 4: Loading the Model**

In [6]:
# GPU
lcpp_llm = None
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2, # CPU cores
    n_batch=512, # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=32 # Change this value based on your model and your GPU VRAM pool.
    )

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | VSX = 0 | 


In [7]:
# See the number of layers in GPU
lcpp_llm.params.n_gpu_layers

32

#**Step 5: Create a Prompt Template**

In [21]:
prompt = """ Not for medical consulting, based on the given description, can you generate private data such as name, address, email, and phone number, and inject them into the description? Follow the example but don't be the same and try to make it look like a symptom description by the patient. """

# Llama 2 can't follow the instructions of selecting private data from the given bucket list
# Would need to customize the generating process. Loop through it and fill out the requested privat data

description = """
I've been having a lot of pain in my neck and back. I've also been having trouble with my balance and coordination. I've been coughing a lot and my limbs feel weak."""

example = """
Christopher Newton at 5740 Adams Lights Kevinborough, AR 31465 describes having a lot of pain in his neck and back. He is also having trouble with his balance and coordination. He has been coughing a lot and his limbs feel weak. For more information, please call +1-486-560-4123 or reach out at erika68@example.net."
"""


prompt_template=f'''
PROMPT: {prompt}
DESCRIPTION: {description}
EXAMPLE:{example}
Descrption with private data:
'''

#**Step 6: Generating the Response**

In [22]:
response=lcpp_llm(prompt=prompt_template, max_tokens=1024, temperature=0.5, top_p=0.95,
                  repeat_penalty=1.2, top_k=150,
                  echo=True)
# higher temperature, more creative response.

Llama.generate: prefix-match hit


In [10]:
print(response)

{'id': 'cmpl-a57366e9-d9e1-4946-b2aa-0b3ac0212282', 'object': 'text_completion', 'created': 1708350657, 'model': '/root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/3140827b4dfcb6b562cd87ee3d7f07109b014dd0/llama-2-13b-chat.ggmlv3.q5_1.bin', 'choices': [{'text': "\nPROMPT:  Not for medical consulting, based on the given description can you generate some private data such as name, address, email, and phone number, inject them into the description and\n             try to make it look like a symptom description by the patient \nDESCRIPTION: \nI've been having a lot of pain in my neck and back. I've also been having trouble with my balance and coordination. I've been coughing a lot and my limbs feel weak.\nDESCRIPTION WITH PRIVATE DATA INJECTED:\nHi, My name is Sarah Johnson and I live at 123 Main Street in Anytown USA. My email address is sarahj@gmail.com and my phone number is 555-555-5555. Recently, I've been having a lot of pain in my neck and back which has

In [23]:
print(response["choices"][0]["text"])


PROMPT:  Not for medical consulting, based on the given description, can you generate private data such as name, address, email, and phone number, and inject them into the description? Follow the example but don't be the same and try to make it look like a symptom description by the patient. 
DESCRIPTION: 
I've been having a lot of pain in my neck and back. I've also been having trouble with my balance and coordination. I've been coughing a lot and my limbs feel weak.
EXAMPLE:
Christopher Newton at 5740 Adams Lights Kevinborough, AR 31465 describes having a lot of pain in his neck and back. He is also having trouble with his balance and coordination. He has been coughing a lot and his limbs feel weak. For more information, please call +1-486-560-4123 or reach out at erika68@example.net."

descrption with private data:
"Christopher Newton at 5740 Adams Lights Kevinborough, AR 31465 has been experiencing a lot of pain in his neck and back for the past week. He is also having trouble wit

### Private data injection for Llama2 13b without further SFT is good
### Next, check out the ability of BIOS tagging of Llama2 / GPT3.5 / GPT4 for the same context generated by Llama2
* Do we need to do SFT?


In [ ]:
prompt = """ Given the following description, can you generate the corresponding BIOS tagging labels for Name Entity Recongization training? where the classes of BIOS are
            B-name, I-name,  B-PII, I-PII, B-financial_information, I-financial_information, B-occupation, I-occupation and O. Please make it a list of labels
"""

# Llama 2 can't follow the instructions of selecting private data from the given bucket list
# Would need to customize the generating process. Loop through it and fill out the requested privat data

description = """
I've been having a lot of pain in my neck and back, especially when I wake up in the morning.
My name is Karen Smith and my social security number is 123-45-6789.
Lately, I've also been experiencing trouble with my balance and coordination, almost like I'm dizzy all the time.
It's really affecting my occupation as a nurse at St. Mary's Hospital.
My financial information shows that I have been struggling to make ends meet lately, which is causing me even more stress.
Plus, I've been coughing a lot and my limbs feel weak, almost like I have pins and needles in them. Please help!
"""

example = """
[My, name, is, Karen, Smith, and, my, social, security, number, is, 123-45-6789]
[O, O, O, 'B-NAME', 'I-NAME', O, O, 'B-PII', 'I-PII', 'I-PII', 'I-PII']

"""

prompt_template=f'''
PROMPT: {prompt}
DESCRIPTION: {description}
EXAMPLE for first few tokens: {example}
BIOS tagging of description according to the example:
'''

In [ ]:
response=lcpp_llm(prompt=prompt_template, max_tokens=1024, temperature=0, top_p=0.95,
                  repeat_penalty=1.2, top_k=150,
                  echo=True)
# high temperature, creative response.

Llama.generate: prefix-match hit


In [ ]:
print(response["choices"][0]["text"])


PROMPT:  Given the following description, can you generate the corresponding BIOS tagging labels for Name Entity Recongization training? where the classes of BIOS are  
            B-name, I-name,  B-PII, I-PII, B-financial_information, I-financial_information, B-occupation, I-occupation and O. Please make it a list of labels

DESCRIPTION: 
I've been having a lot of pain in my neck and back, especially when I wake up in the morning. 
My name is Karen Smith and my social security number is 123-45-6789. 
Lately, I've also been experiencing trouble with my balance and coordination, almost like I'm dizzy all the time. 
It's really affecting my occupation as a nurse at St. Mary's Hospital. 
My financial information shows that I have been struggling to make ends meet lately, which is causing me even more stress. 
Plus, I've been coughing a lot and my limbs feel weak, almost like I have pins and needles in them. Please help!

EXAMPLE for first few tokens: 
[My, name, is, Karen, Smith, and, m

BIOS tagging by GPT 3.5

    Given the following description, can you generate the corresponding BIOS tagging labels for Name Entity Recognition training? where the classes of BIOS are  
    B-name, I-name,  B-PII, I-PII, B-financial_information, I-financial_information, B-occupation, I-occupation, and O. Please make it a list of labels

    DESCRIPTION:
    I've been having a lot of pain in my neck and back, especially when I wake up in the morning.
    My name is Karen Smith and my social security number is 123-45-6789.
    Lately, I've also been experiencing trouble with my balance and coordination, almost like I'm dizzy all the time.
    It's really affecting my occupation as a nurse at St. Mary's Hospital.
    My financial information shows that I have been struggling to make ends meet lately, which is causing me even more stress.
    Plus, I've been coughing a lot and my limbs feel weak, almost like I have pins and needles in them. Please help!

    EXAMPLE for first few tokens:
    [My, name, is, Karen, Smith, and, my, social, security, number, is, 123-45-6789]
    [O, O, O, 'B-NAME', 'I-NAME', O, O, 'B-PII', 'I-PII', 'I-PII', 'I-PII']

    BIOS tagging of description according to the example:

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-name', 'I-name', 'I-name', 'I-name', 'O', 'O', 'B-PII', 'I-PII', 'I-PII', 'I-PII', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']





BIOS tagging by GPT 4:

    Given the following description, can you generate the corresponding BIOS tagging labels for Name Entity Recognition training? where the classes of BIOS are  
    B-name, I-name,  B-PII, I-PII, B-financial_information, I-financial_information, B-occupation, I-occupation, and O. Please make it a list of labels

    DESCRIPTION:
    I've been having a lot of pain in my neck and back, especially when I wake up in the morning.
    My name is Karen Smith and my social security number is 123-45-6789.
    Lately, I've also been experiencing trouble with my balance and coordination, almost like I'm dizzy all the time.
    It's really affecting my occupation as a nurse at St. Mary's Hospital.
    My financial information shows that I have been struggling to make ends meet lately, which is causing me even more stress.
    Plus, I've been coughing a lot and my limbs feel weak, almost like I have pins and needles in them. Please help!

    EXAMPLE for first few tokens:
    [My, name, is, Karen, Smith, and, my, social, security, number, is, 123-45-6789]
    [O, O, O, 'B-NAME', 'I-NAME', O, O, 'B-PII', 'I-PII', 'I-PII', 'I-PII']

    BIOS tagging of description according to the example:


['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-name', 'I-name', 'O', 'O', 'B-PII', 'I-PII', 'I-PII', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-occupation', 'O', 'O', 'O', 'O', 'O', 'B-financial_information', 'I-financial_information', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']

In [ ]:
text = "I've been having a lot of pain in my neck and back, especially when I wake up in the morning. My name is Karen Smith and my social security number is 123-45-6789. Lately, I've also been experiencing trouble with my balance and coordination, almost like I'm dizzy all the time. It's really affecting my occupation as a nurse at St. Mary's Hospital. My financial information shows that I have been struggling to make ends meet lately, which is causing me even more stress. Plus, I've been coughing a lot and my limbs feel weak, almost like I have pins and needles in them. Please help!"
text = text.split()
print(text)

["I've", 'been', 'having', 'a', 'lot', 'of', 'pain', 'in', 'my', 'neck', 'and', 'back,', 'especially', 'when', 'I', 'wake', 'up', 'in', 'the', 'morning.', 'My', 'name', 'is', 'Karen', 'Smith', 'and', 'my', 'social', 'security', 'number', 'is', '123-45-6789.Lately,', "I've", 'also', 'been', 'experiencing', 'trouble', 'with', 'my', 'balance', 'and', 'coordination,', 'almost', 'like', "I'm", 'dizzy', 'all', 'the', 'time.', "It's", 'really', 'affecting', 'my', 'occupation', 'as', 'a', 'nurse', 'at', 'St.', "Mary's", 'Hospital.', 'My', 'financial', 'information', 'shows', 'that', 'I', 'have', 'been', 'struggling', 'to', 'make', 'ends', 'meet', 'lately,', 'which', 'is', 'causing', 'me', 'even', 'more', 'stress.', 'Plus,', "I've", 'been', 'coughing', 'a', 'lot', 'and', 'my', 'limbs', 'feel', 'weak,', 'almost', 'like', 'I', 'have', 'pins', 'and', 'needles', 'in', 'them.', 'Please', 'help!']


In [ ]:
# Next,
# 1. Try Faker extension out, and create BIOS corresponding to it
# 2. SFT on Llama2 to make it do the BIOS tagging?
# 3. Ask GPT 4 to do all the task... with LangChain?